# Proposition 1: New Hires

This SQL query identifies employees who also have job candidate records in the `HumanResources.JobCandidate` table. It uses a Common Table Expression (CTE) named `newEmps` to find the intersection between `BusinessEntityID` values in both the `HumanResources.JobCandidate` and `HumanResources.Employee` tables. It then retrieves all columns from the `HumanResources.Employee` table for these matched employees.

### Key Objectives:

1. **Identify Employees with Job Candidate Records**: The query selects employees who are listed in both the `HumanResources.JobCandidate` and `HumanResources.Employee` tables by comparing their `BusinessEntityID`.
2. **Retrieve Employee Information**: For each matched employee, the query returns all information from the `HumanResources.Employee` table.
3. **Efficient Querying with CTE and INTERSECT**: The use of the CTE and `INTERSECT` ensures that only employees who appear in both tables are considered.

## Functional Specification

### Inputs

- **HumanResources.JobCandidate table**: Contains records of job candidates, identified by `BusinessEntityID`.
- **HumanResources.Employee table**: Contains employee records, identified by `BusinessEntityID`.

### Outputs

- **All columns from `HumanResources.Employee`**: The query retrieves all available columns for the employees who are also listed in the `JobCandidate` table. This could include columns like `BusinessEntityID`, `JobTitle`, `HireDate`, etc.

### Query Breakdown

1. **Common Table Expression (CTE)**:
    
    - The CTE `newEmps` is used to store the result of an `INTERSECT` operation.
    - It selects the `BusinessEntityID` values that are common to both the `HumanResources.JobCandidate` and `HumanResources.Employee` tables, ensuring that only entities that exist in both tables are considered.
2. **INTERSECT Operation**:
    
    - The `INTERSECT` keyword is used to find the common `BusinessEntityID` values between the two tables.
    - This is efficient because it avoids the need for an explicit join between these two tables; instead, it directly identifies shared records based on `BusinessEntityID`.
3. **Select Employee Information**:
    
- After the CTE has identified the employees who are also job candidates, the main query retrieves all columns from the `HumanResources.Employee` table for these employees by performing a join on `BusinessEntityID`.
- The `WHERE` clause ensures that only employees whose `BusinessEntityID` exists in the `newEmps` CTE are selected.

### Edge Cases

- **No Matching Records**: If no employees have corresponding job candidate records, the `INTERSECT` will return no rows, and the main query will produce an empty result set.
- **Duplicate Entries**: The `INTERSECT` inherently avoids duplicates, so even if a `BusinessEntityID` appears multiple times in either table, it will only be returned once in the `newEmps` CTE.
- **Missing Data in Either Table**: If an employee does not have a corresponding record in the `JobCandidate` table (or vice versa), they will not be included in the result set.

### Assumptions

- **Unique BusinessEntityID**: It is assumed that `BusinessEntityID` is unique in both the `HumanResources.Employee` and `HumanResources.JobCandidate` tables.
- **Complete Data**: The query assumes that the `HumanResources.Employee` table contains all relevant employee data and that any employee appearing in `JobCandidate` who is not also in `Employee` is irrelevant for the purpose of this query.

### Performance Considerations

- **Indexing**: Indexes on the `BusinessEntityID` column in both the `HumanResources.JobCandidate` and `HumanResources.Employee` tables will enhance the performance of this query, especially given the use of the `INTERSECT` operator.

In [23]:
WITH newEmps AS
(
        SELECT BusinessEntityID
        FROM HumanResources.JobCandidate
    INTERSECT
        SELECT BusinessEntityID
        FROM HumanResources.Employee
)
SELECT e.*
FROM newEmps, HumanResources.Employee AS e
WHERE e.BusinessEntityID = newEmps.BusinessEntityID;

(2 rows affected)

Total execution time: 00:00:00.006

BusinessEntityID,NationalIDNumber,LoginID,OrganizationNode,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
212,885055826,adventure-works\peng0,0x7B56,3,Quality Assurance Supervisor,1976-03-18,M,M,2008-12-09,1,81,60,1,e249d613-36c9-4544-9b6f-6ce50e5e0da5,2014-06-30 00:00:00.000
274,502097814,adventure-works\stephen0,0x9560,2,North American Sales Manager,1951-10-17,M,M,2011-01-04,1,14,27,1,86d5fa9e-4bc0-48c8-91dc-1ec467418d11,2014-06-30 00:00:00.000


# Proposition 2: Sales Demographic

This SQL query combines data from two different sources (`Sales.Customer` and `Sales.SalesPerson`) to count how many people (either customers or salespersons) are associated with each sales territory. It returns the territory name and the total count of people associated with each territory, sorted in descending order by the count.

### Key Objectives:

1. **Merge Data from Multiple Sources**: The query combines territory information from both `Sales.Customer` and `Sales.SalesPerson` using a `UNION ALL` operation.
2. **Count People per Territory**: It counts how many records (representing either customers or salespersons) are associated with each sales territory.
3. **Display Territory Names**: The result includes the name of each sales territory, and the output is ordered by the count of people per territory in descending order.

## Functional Specification

### Inputs

- **Sales.Customer**: Contains customer records, including a `TerritoryID` for each customer, identifying the sales territory they belong to.
- **Sales.SalesPerson**: Contains salesperson records, also with a `TerritoryID`, identifying the sales territory they operate in.
- **Sales.SalesTerritory**: Contains information about sales territories, including `TerritoryID` and `Name` (the name of the territory).

### Outputs

- **Name**: The name of the sales territory.
- **PepCount**: The count of customers and salespersons associated with that territory.

### Query Breakdown

1. **Common Table Expression (CTE) - `SalesDemo`**:
    
    - The query starts by creating a CTE called `SalesDemo`, which selects the `TerritoryID` from both the `Sales.Customer` and `Sales.SalesPerson` tables using `UNION ALL`.
    - `UNION ALL` ensures that all records from both tables are included, even if duplicates exist (i.e., a customer and a salesperson can belong to the same territory).
2. **Counting People in Territories**:
    
    - After combining the `TerritoryID` data, the query joins this result (`SalesDemo`) with the `Sales.SalesTerritory` table on `TerritoryID` to retrieve the name of each territory.
    - The query then counts how many people (customers and salespersons combined) are associated with each territory using `COUNT(SD.TerritoryID)`.
3. **Grouping and Ordering**:
    
- The results are grouped by `TerritoryID` and `Name`, ensuring that each row in the output corresponds to a distinct sales territory.
- The final result is sorted by the count of people (`PepCount`) in descending order to show the most populous territories first.

### Edge Cases

- **Territories with No People**: If a territory has no customers or salespersons, it will not appear in the final result set, as only territories with a `TerritoryID` in `Sales.Customer` or `Sales.SalesPerson` are included.
- **Duplicate Territory IDs**: The `UNION ALL` ensures that if a territory has both customers and salespersons, all of them will be counted, as duplicates are not removed.

### Assumptions

- The `TerritoryID` field is a common key across the three tables (`Sales.Customer`, `Sales.SalesPerson`, and `Sales.SalesTerritory`).
- The tables are indexed on `TerritoryID` to optimize the performance of the join operation and the aggregation.

### Considerations

- **Performance**: Depending on the volume of data, the `UNION ALL` operation may cause performance issues if there are many duplicate records. Indexing and possibly limiting the data retrieved may be necessary for large datasets.
- **Completeness of Data**: The assumption is that every `TerritoryID` in `Sales.Customer` and `Sales.SalesPerson` has a corresponding entry in `Sales.SalesTerritory`.

In [24]:
WITH SalesDemo AS
(
        SELECT TerritoryID
        FROM Sales.Customer
    UNION ALL
        SELECT TerritoryID
        FROM Sales.SalesPerson
)
SELECT ST.Name, COUNT(SD.TerritoryID) AS PepCount
FROM SalesDemo AS SD
    INNER JOIN Sales.SalesTerritory AS ST
        ON SD.TerritoryID = ST.TerritoryID
GROUP BY SD.TerritoryID, ST.Name
ORDER BY PepCount DESC;

(10 rows affected)

Total execution time: 00:00:00.006

Name,PepCount
Southwest,4698
Australia,3666
Northwest,3523
United Kingdom,1992
France,1885
Germany,1853
Canada,1793
Southeast,177
Central,133
Northeast,114


# Proposition 3: Diversity Check

The SQL query retrieves a list of people from the `Person.Person` table who have a non-null title, excluding those with the title 'Mr.'. The query uses the `EXCEPT` operator to eliminate rows with 'Mr.' from the final result set.

### Key Objectives:

1. **Filter by Non-Null Titles**: Retrieve records where the `Title` column is not `NULL`.
2. **Exclude 'Mr.' Title**: Exclude records where the title is 'Mr.' from the result set.
3. **Return Person Information**: The query selects key information such as `Title`, `FirstName`, `MiddleName`, `LastName`, and `Suffix`.

## Functional Specification

### Inputs

- **Person.Person table**: Contains personal details for each individual, including `Title`, `FirstName`, `MiddleName`, `LastName`, and `Suffix`.

### Outputs

- **Title**: The title of the person (e.g., 'Ms.', 'Dr.', etc.), excluding 'Mr.'.
- **FirstName**: The first name of the person.
- **MiddleName**: The middle name of the person (if available).
- **LastName**: The last name (surname) of the person.
- **Suffix**: Any suffix attached to the person’s name (e.g., 'Jr.', 'Sr.').

### Query Breakdown

1. **Filter Non-Null Titles**:
    
    - The first `SELECT` statement retrieves all records where the `Title` is not `NULL`. This ensures that only records with titles are included in the result set.
2. **Exclude 'Mr.' Title**:
    
    - The `EXCEPT` operator is used to exclude any records where the `Title` is 'Mr.' from the final result set.
    - The second `SELECT` statement identifies all records where the `Title` is 'Mr.' and excludes them from the result set generated by the first `SELECT` statement.
3. **Return Specific Columns**:
    
- The query selects specific columns: `Title`, `FirstName`, `MiddleName`, `LastName`, and `Suffix`, for individuals that meet the filtering criteria.

### Edge Cases

- **No Title Assigned**: People without a title (i.e., where `Title` is `NULL`) will not appear in the result set because of the `WHERE Title IS NOT NULL` clause.
- **Titles Other than 'Mr.'**: All titles other than 'Mr.' will appear in the result set, assuming they are not `NULL`.
- **People with MiddleName or Suffix as `NULL`**: These fields are optional, so they may appear as `NULL` for certain individuals in the result set, but this does not affect the selection of rows.

### Assumptions

- The `Person.Person` table contains a variety of titles besides 'Mr.' (e.g., 'Ms.', 'Dr.', etc.).
- The `EXCEPT` operator is used to ensure that records where `Title = 'Mr.'` are excluded from the result set, effectively filtering out such rows from the initial query.

### Performance Considerations

- **Indexes**: Proper indexing on the `Title` column may improve query performance, especially when dealing with large datasets.
- **NULL Handling**: Ensure that the database correctly handles `NULL` values in the `Title` field to avoid unexpected behavior.

This specification outlines the intent and functionality of the SQL query, highlighting its purpose of filtering and excluding specific rows based on the `Title` field.

In [25]:
    SELECT Title, FirstName, MiddleName, LastName, Suffix
    FROM Person.Person
    WHERE Title IS NOT NULL
EXCEPT
    SELECT Title, FirstName, MiddleName, LastName, Suffix
    FROM Person.Person
    WHERE Title = 'Mr.';

(432 rows affected)

Total execution time: 00:00:00.054

Title,FirstName,MiddleName,LastName,Suffix
Ms.,Alice,M.,Steiner,NULL
Ms.,Becky,R.,Waters,NULL
Ms.,Caroline,M.,Woodard,NULL
Ms.,Cheryl,J.,Pompa,NULL
Ms.,Denean,J.,Ison,NULL
Ms.,Fran,P.,Highfill,NULL
Ms.,Gail,A,Erickson,NULL
Ms.,Janice,M,Galvin,NULL
Ms.,Joanna,B.,Wall,NULL
Ms.,Julia,NULL,Moseley,NULL


# Proposition 4: 2011 Long-Term Employees

The SQL query retrieves a list of employees who started working in a department during the year 2011 but did not end their employment within the same year. Specifically, the query looks for records in the `HumanResources.EmployeeDepartmentHistory` table where the `StartDate` is between January 1, 2011, and January 1, 2012, excluding any records where the `EndDate` falls within the same date range.

### Key Objectives:

1. **Filter by Start Date**: Identify employees who started their work in a department within the year 2011.
2. **Exclude by End Date**: Exclude employees who ended their department employment within the same period (2011).
3. **Return the Results**: Display the `BusinessEntityID`, `StartDate`, and `EndDate` for qualifying records.

## Functional Specification

### Inputs

- **HumanResources.EmployeeDepartmentHistory**: A table containing records of employee department history, including:
    - `BusinessEntityID`: Unique identifier for the employee.
    - `StartDate`: The date when the employee started in a particular department.
    - `EndDate`: The date when the employee ended their employment in that department.

### Outputs

- **BusinessEntityID**: The unique identifier of the employee.
- **StartDate**: The date when the employee started in the department.
- **EndDate**: The date when the employee ended their department assignment (this may be `NULL` if still employed in that department).

### Query Breakdown

1. **Initial Selection (StartDate)**:
    
    - The first part of the query selects records from `HumanResources.EmployeeDepartmentHistory` where the `StartDate` falls between January 1, 2011, and January 1, 2012. This includes employees who began their employment within this period.
2. **Exclusion (EndDate)**:
    
    - The `EXCEPT` clause removes any records where the `EndDate` also falls within the same range (January 1, 2011, to January 1, 2012). This ensures that only employees who did **not** end their employment in the same year are returned.
3. **Result**:
    
- The query returns the `BusinessEntityID`, `StartDate`, and `EndDate` for employees who meet the conditions.

### Edge Cases

- **Employees without an End Date**: If an employee's `EndDate` is `NULL`, they are not excluded by the second part of the query and will appear in the results if their `StartDate` falls within 2011.
- **Employees Who Start and End on the Same Date**: If an employee both starts and ends their employment on the same date within the specified range, they will be excluded from the result set due to the `EXCEPT` clause.

### Assumptions

- The `HumanResources.EmployeeDepartmentHistory` table is properly indexed on `BusinessEntityID`, `StartDate`, and `EndDate` to ensure performance.
- The goal of the query is to identify employees who remained in their department beyond 2011 and exclude those who both started and ended within the same year.

In [26]:
    SELECT BusinessEntityID, StartDate, EndDate
    FROM HumanResources.EmployeeDepartmentHistory
    WHERE StartDate BETWEEN '2011-01-01' AND '2012-01-01'
EXCEPT
    SELECT BusinessEntityID, StartDate, EndDate
    FROM HumanResources.EmployeeDepartmentHistory
    WHERE EndDate BETWEEN '2011-01-01' AND '2012-01-01';

(17 rows affected)

Total execution time: 00:00:00.010

BusinessEntityID,StartDate,EndDate
15,2011-01-18,NULL
18,2011-02-07,NULL
19,2011-02-14,NULL
20,2011-01-07,NULL
224,2011-09-01,NULL
250,2011-07-31,2012-07-14
273,2011-02-15,NULL
274,2011-01-04,NULL
275,2011-05-31,NULL
276,2011-05-31,NULL


# Proposition 5: Happy Hour Party Invites For 5+ Years of Employment and High Value Employees

The SQL query is designed to retrieve employee details (such as `LoginID`, `JobTitle`, `OrganizationLevel`, `BirthDate`, and `HireDate`) based on specific filtering conditions. The query returns employees who:

1. Were born before January 1, 1993.
2. Were hired before January 1, 2009.
3. Have an `OrganizationLevel` of 3 or higher.

The final result set is ordered by `OrganizationLevel` and `JobTitle`.

### Key Objectives:

1. **Filter Employees by Birth Date**: Select employees who were born before January 1, 1993.
2. **Exclude Employees by Hire Date**: Exclude employees who were hired on or after January 1, 2009.
3. **Filter by Organization Level**: Select only those employees whose `OrganizationLevel` is 3 or higher.
4. **Sort Results**: The results are ordered by `OrganizationLevel` and `JobTitle` in ascending order.

## Functional Specification

### Inputs

- **HumanResources.Employee table**: Contains employee records, including `LoginID`, `JobTitle`, `OrganizationLevel`, `BirthDate`, and `HireDate`.

### Outputs

- **LoginID**: The unique login identifier for the employee.
- **JobTitle**: The employee’s job title within the organization.
- **OrganizationLevel**: The level of the employee in the organizational hierarchy (with higher numbers indicating higher levels).
- **BirthDate**: The employee’s birthdate, used to filter by age.
- **HireDate**: The date the employee was hired, used to filter based on employment duration.

### Query Breakdown

1. **Select Employees Born Before 1993**:
    
- The first subquery selects employees whose `BirthDate` is earlier than `'1993-01-01'`.
3. **Exclude Employees Hired After 2009**:
    
- The `EXCEPT` clause removes employees from the first subquery who were hired on or after `'2009-01-01'`.
5. **Include Only Employees at Organization Level 3 or Higher**:
    
- The `INTERSECT` clause ensures that the final result set includes only employees whose `OrganizationLevel` is greater than or equal to 3.
7. **Order the Results**:
    
- The final result set is ordered by `OrganizationLevel` and `JobTitle` to provide a structured view of the employees.

### Edge Cases

- **No Matching Employees**: If no employees meet all the conditions, the result set will be empty.
- **Employees Hired on Exactly January 1, 2009**: Employees hired on this date will be excluded due to the `HireDate >= '2009-01-01'` condition.
- **Organization Levels Below 3**: Any employees with an `OrganizationLevel` less than 3 will be excluded, even if they satisfy the birth and hire date conditions.

### Assumptions

- The `HumanResources.Employee` table contains accurate and up-to-date information about employee birthdates, hire dates, and organizational levels.
- The business requirement is to filter by birthdate, exclude based on hire date, and include only employees at or above a specific organization level.

In [27]:
(
    SELECT LoginID, JobTitle, OrganizationLevel, BirthDate, HireDate
    FROM HumanResources.Employee
    WHERE BirthDate < '1993-01-01'
EXCEPT
    SELECT LoginID, JobTitle, OrganizationLevel, BirthDate, HireDate
    FROM HumanResources.Employee
    WHERE HireDate >= '2009-01-01'
INTERSECT
    SELECT LoginID, JobTitle, OrganizationLevel, BirthDate, HireDate
    FROM HumanResources.Employee
    WHERE OrganizationLevel >= 3
)
ORDER BY OrganizationLevel, JobTitle;

(104 rows affected)

Total execution time: 00:00:00.011

LoginID,JobTitle,OrganizationLevel,BirthDate,HireDate
adventure-works\ken0,Chief Executive Officer,NULL,1969-01-29,2009-01-14
adventure-works\laura1,Chief Financial Officer,1,1976-01-06,2009-01-31
adventure-works\jean0,Information Services Manager,1,1975-12-13,2008-12-11
adventure-works\david0,Marketing Manager,1,1975-03-19,2007-12-20
adventure-works\terri0,Vice President of Engineering,1,1971-08-01,2008-01-31
adventure-works\james1,Vice President of Production,1,1983-01-07,2009-02-03
adventure-works\brian3,Vice President of Sales,1,1977-06-06,2011-02-15
adventure-works\david6,Accounts Manager,2,1983-07-08,2009-01-30
adventure-works\dan0,Application Specialist,2,1987-05-26,2009-01-11
adventure-works\janaina0,Application Specialist,2,1985-01-30,2008-12-23


# Proposition 6: Married vs Single Employees Vacation Time

The SQL query calculates the average number of vacation hours for employees based on their marital status ('M' for married and 'S' for single). It first retrieves the vacation hours for employees with the specified marital status and then computes the average vacation hours for each group.

### Key Objectives:

1. **Group Employees by Marital Status**: The query focuses on employees who are either married ('M') or single ('S').
2. **Calculate Average Vacation Hours**: The query calculates the average number of vacation hours for both married and single employees.
3. **Use of UNION ALL**: The query uses `UNION ALL` to combine results from two different subsets of data (married and single employees) into one result set.

## Functional Specification

### Inputs

- **HumanResources.Employee table**: Contains employee records, including `MaritalStatus` and `VacationHours`.

### Outputs

- **MaritalStatus**: The marital status of the employees ('M' for married and 'S' for single).
- **AverageVacationHours**: The calculated average of vacation hours for employees in each marital status group.

### Query Breakdown

1. **Subquery for Sorting**:
    
    - The query uses a subquery to assign a constant sort value to distinguish between married (`sort = 1`) and single (`sort = 2`) employees.
    - This is done using the `UNION ALL` operator to retrieve the vacation hours for both groups of employees.
2. **Vacation Hours Calculation**:
    
    - For each marital status ('M' and 'S'), the query calculates the total vacation hours using `SUM(VacationHours)`.
    - It then divides this total by the count of employees within the group (`COUNT(sort)`) to compute the average vacation hours.
3. **Grouping**:
    
- The query groups results by both the `sort` column (to separate married and single employees) and `MaritalStatus`.

### Logic

- **First Subquery**: Selects all married employees (`MaritalStatus = 'M'`) and assigns `sort = 1` for identification.
- **Second Subquery**: Selects all single employees (`MaritalStatus = 'S'`) and assigns `sort = 2`.
- **`UNION ALL`**: Combines both result sets into a single table for processing.
- **`GROUP BY sort, MaritalStatus`**: Ensures that results are grouped by marital status and sorted by the `sort` value to keep results for married and single employees distinct.

### Outputs

- The query returns two rows: one for married employees and one for single employees.
- Each row will include the marital status and the calculated average vacation hours for that group.

### Edge Cases

- **No Employees with a Specific Marital Status**: If there are no employees with either marital status ('M' or 'S'), the query will still return the other group's result.
- **Zero Vacation Hours**: If an employee has zero vacation hours, it will be included in the calculation, which could lower the average.

### Assumptions

- **Data Completeness**: All employees have valid `VacationHours` and `MaritalStatus` values.
- **Marital Status Codes**: The only marital statuses considered are 'M' (married) and 'S' (single), and other statuses are not included.

In [28]:
SELECT MaritalStatus, SUM(VacationHours)/COUNT(sort) AS AverageVacationHours
FROM (SELECT 1 AS sort, MaritalStatus, VacationHours
        FROM HumanResources.Employee
        WHERE MaritalStatus = 'M'
    UNION ALL
        SELECT 2, MaritalStatus, VacationHours
        FROM HumanResources.Employee
        WHERE MaritalStatus = 'S') AS E
GROUP BY sort, MaritalStatus;

(2 rows affected)

Total execution time: 00:00:00.007

MaritalStatus,AverageVacationHours
M,48
S,53


# Proposition 7: Recently Promoted Employees

This SQL query retrieves a list of employees who have both received a pay rate change since January 1, 2013, and have opted to receive email promotions. The query selects specific details about these employees, including their `BusinessEntityID`, `LoginID`, `JobTitle`, and `HireDate`.

### Key Objectives:

1. **Identify Recent Promotions**: The query identifies employees who have had a pay rate change on or after January 1, 2013.
2. **Filter by Email Promotion**: The query further narrows down this list to employees who have opted into receiving email promotions.
3. **Return Employee Details**: For the filtered employees, the query retrieves basic employee information from the `HumanResources.Employee` table.
4. **Use of Common Table Expression (CTE)**: The query uses a CTE (`RecentPromotions`) to simplify the filtering logic before joining with the `HumanResources.Employee` table.

## Functional Specification

### Inputs

- **HumanResources.EmployeePayHistory table**: Contains employee pay history records, including `BusinessEntityID` and `RateChangeDate`.
- **Person.Person table**: Contains personal information of individuals, including `BusinessEntityID` and their preferences for email promotions (`EmailPromotion`).
- **HumanResources.Employee table**: Contains employee details, including `BusinessEntityID`, `LoginID`, `JobTitle`, and `HireDate`.

### Outputs

- **BusinessEntityID**: The unique identifier for the employee.
- **LoginID**: The login ID associated with the employee.
- **JobTitle**: The job title of the employee.
- **HireDate**: The date when the employee was hired.

### Query Breakdown

1. **Common Table Expression (CTE) – `RecentPromotions`**:
    
    - The `RecentPromotions` CTE is used to create a set of employees who meet **both** of the following conditions:
        - They have had a pay rate change on or after January 1, 2013.
        - They have opted to receive email promotions (where `EmailPromotion` is greater than 0).
    - The CTE uses the `INTERSECT` operation to return only those employees (`BusinessEntityID`) who are found in both the `HumanResources.EmployeePayHistory` and the `Person.Person` tables with the specified conditions.
2. **Main Query**:
    
- The main query selects employee details (`BusinessEntityID`, `LoginID`, `JobTitle`, and `HireDate`) from the `HumanResources.Employee` table.
- It joins the results of the `RecentPromotions` CTE with the `HumanResources.Employee` table on `BusinessEntityID` to return only those employees who meet the promotion and email conditions.

### Explanation of SQL Clauses

1. **WITH Clause (CTE)**:
    
    - The `WITH` clause creates a temporary result set (`RecentPromotions`) by intersecting two sets of employees:
        - Employees who have had a pay rate change since 2013 (from `HumanResources.EmployeePayHistory`).
        - Employees who have opted into email promotions (from `Person.Person`).
2. **INTERSECT**:
    
    - The `INTERSECT` operation is used to find the common `BusinessEntityID`s between the two result sets (i.e., those who satisfy both conditions).
3. **Main SELECT Statement**:
    
    - The main query selects employee details from `HumanResources.Employee` by matching the `BusinessEntityID` from the `RecentPromotions` CTE to that in the `HumanResources.Employee` table.
4. **Filtering**:
    
    - The `WHERE` clause ensures that only employees present in the `RecentPromotions` CTE (those who meet both conditions) are included in the final result set.

### Edge Cases

- **No Matching Employees**: If no employees meet both conditions (i.e., recent pay rate change and opted for email promotion), the query will return an empty result set.
- **Multiple Rate Changes**: If an employee has multiple rate changes after January 1, 2013, they will still be included, as the query does not limit based on the number of rate changes.

### Assumptions

- Employees with `EmailPromotion = 0` are assumed to have opted out of email promotions.
- The query assumes that both `HumanResources.EmployeePayHistory` and `Person.Person` tables are up-to-date and contain valid data for employee promotions and email preferences.
- The `INTERSECT` operation is used to ensure that only employees who meet **both** criteria (recent promotion and email preference) are included in the result.

In [29]:
WITH RecentPromotions AS
(
    SELECT BusinessEntityID
    FROM HumanResources.EmployeePayHistory
    WHERE RateChangeDate >= '2013-01-01'
INTERSECT
    SELECT BusinessEntityID
    FROM Person.Person
    WHERE EmailPromotion > 0
)
SELECT E.BusinessEntityID, E.LoginID, E.JobTitle, E.HireDate
FROM RecentPromotions AS RP, HumanResources.Employee AS E
WHERE RP.BusinessEntityID = E.BusinessEntityID;

(4 rows affected)

Total execution time: 00:00:00.003

BusinessEntityID,LoginID,JobTitle,HireDate
172,adventure-works\marc0,Production Technician - WC30,2009-01-16
174,adventure-works\benjamin0,Production Technician - WC30,2009-01-27
178,adventure-works\john3,Production Technician - WC30,2009-03-03
286,adventure-works\lynn0,Sales Representative,2013-05-30


# Proposition 8: The D's

This SQL query retrieves a distinct list of people from the `Person.Person` table, based on specific name criteria, and orders the results by the last name. The query combines data using `EXCEPT` and `UNION` to achieve the desired results.

### Key Objectives:

1. **Filter Based on First, Last, and Middle Names**:
    - Select people whose first name starts with 'D', excluding those whose last name starts with 'D'.
    - Include those whose middle name starts with 'D', even if the first or last name does not match the condition.
2. **Order by Last Name**: The final result is sorted alphabetically by the `LastName`.

## Functional Specification

### Inputs

- **Person.Person table**: Contains personal details including `Title`, `FirstName`, `MiddleName`, `LastName`, and `Suffix`.

### Outputs

- **Title**: Title of the person (e.g., Mr., Ms., Dr.).
- **FirstName**: First name of the person.
- **MiddleName**: Middle name of the person.
- **LastName**: Last name of the person.
- **Suffix**: Any suffix associated with the person's name (e.g., Jr., Sr., III).

### Query Breakdown

1. **First Name Starting with 'D'**:
    
    - The first `SELECT` statement retrieves all rows where the `FirstName` starts with the letter 'D'. This is done using the `LIKE N'D%'` clause.
2. **Exclude Last Name Starting with 'D'**:
    
    - The `EXCEPT` clause excludes rows where the `LastName` starts with the letter 'D', ensuring that any person whose last name begins with 'D' is removed from the result.
3. **Include Middle Name Starting with 'D'**:
    
    - The `UNION` operator adds to the result set any rows where the `MiddleName` starts with the letter 'D', regardless of whether their first or last name starts with 'D'.
4. **Order by Last Name**:
    
- Finally, the result is ordered by the `LastName` in ascending alphabetical order.

### Logic Breakdown

- **EXCEPT**: This operator removes the rows from the first result set (`FirstName LIKE 'D%'`) that appear in the second result set (`LastName LIKE 'D%'`).
- **UNION**: This operator merges the result of the first condition (after the `EXCEPT` operation) with another result set that matches `MiddleName LIKE 'D%'`. The `UNION` removes any duplicates.

### Outputs

The query returns the following fields for individuals meeting the conditions:

- **Title**: Formal title of the person.
- **FirstName**: First name starting with 'D'.
- **MiddleName**: Middle name starting with 'D' (if present).
- **LastName**: Last name of the person (not starting with 'D', in cases selected using `EXCEPT`).
- **Suffix**: Any name suffix, such as Jr., Sr., or III.

### Edge Cases

- **Empty Fields**: If `MiddleName` or `Suffix` is `NULL` or not available, these fields will be returned as `NULL`.
- **Name Conflicts**: If a person’s name matches more than one condition (e.g., both `FirstName` and `MiddleName` start with 'D'), duplicates are removed because of the `UNION` operation.
- **No Matches**: If no names match the specified conditions, the query will return an empty result set.

### Assumptions

- The `Person.Person` table is indexed appropriately for the `FirstName`, `LastName`, and `MiddleName` fields for optimal performance.
- The business requirement allows excluding people whose `LastName` starts with 'D' but permits including people whose `MiddleName` starts with 'D'.

In [30]:
(   SELECT Title, FirstName, MiddleName, LastName, Suffix
    FROM Person.Person
    WHERE FirstName LIKE N'D%'
EXCEPT
    SELECT Title, FirstName, MiddleName, LastName, Suffix
    FROM Person.Person
    WHERE LastName LIKE N'D%'
UNION
    SELECT Title, FirstName, MiddleName, LastName, Suffix
    FROM Person.Person
    WHERE MiddleName LIKE N'D%')
ORDER BY LastName;

(1713 rows affected)

Total execution time: 00:00:00.054

Title,FirstName,MiddleName,LastName,Suffix
NULL,Devin,NULL,Adams,NULL
NULL,Dalton,NULL,Adams,NULL
NULL,Mary,D,Adams,NULL
NULL,Jeremy,D,Adams,NULL
NULL,Dylan,NULL,Alexander,NULL
NULL,Destiny,NULL,Alexander,NULL
NULL,David,NULL,Alexander,NULL
NULL,Dalton,NULL,Alexander,NULL
NULL,Ian,D,Alexander,NULL
NULL,Devin,NULL,Alexander,NULL


# Proposition 9: Recently Raised Pay History

The SQL query identifies employees who have both changed departments (based on having a non-NULL `EndDate` in their department history) and received a raise (based on a pay rate change on or after January 1, 2012). The query retrieves all pay history records for these employees and orders them by `BusinessEntityID` and `RateChangeDate`.

### Key Objectives:

1. **Identify Employees with Department Changes**: Select employees who have had a department change (i.e., their `EndDate` in the `EmployeeDepartmentHistory` table is not `NULL`).
2. **Filter by Recent Pay Raises**: Select employees who have received a pay rate change on or after January 1, 2012, from the `EmployeePayHistory` table.
3. **Combine Results**: Retrieve all pay history records for these employees and display them ordered by employee and rate change date.

## Functional Specification

### Inputs

- **HumanResources.EmployeeDepartmentHistory table**: Contains department history records for employees, including `BusinessEntityID` and `EndDate`.
- **HumanResources.EmployeePayHistory table**: Contains pay history records for employees, including `BusinessEntityID`, `RateChangeDate`, and other pay-related fields.

### Outputs

- All columns from the `HumanResources.EmployeePayHistory` table for employees who meet the department change and recent raise criteria.

### Query Breakdown

1. **Common Table Expression (CTE) - RecentRaises**:
    
    - This section of the query uses a CTE to identify employees who:
        - Had a department change (where `EndDate` in `EmployeeDepartmentHistory` is not `NULL`).
        - Received a pay rate change on or after January 1, 2012 (using the `RateChangeDate` from `EmployeePayHistory`).
    - The `INTERSECT` operator ensures that only employees who satisfy **both** criteria (department change and recent pay raise) are included.
2. **Main Query**:
    
    - The main query selects all columns from the `HumanResources.EmployeePayHistory` table for the employees identified in the `RecentRaises` CTE.
    - It joins the `RecentRaises` CTE to `EmployeePayHistory` on the `BusinessEntityID` to get detailed pay history records for each employee.
3. **Ordering**:
    
- The results are ordered by `BusinessEntityID` (to group records by employee) and by `RateChangeDate` (to display the pay changes in chronological order for each employee).

### Edge Cases

- **No Department Changes**: If no employees have a `NULL` `EndDate` in `EmployeeDepartmentHistory`, the query will return no results.
- **No Recent Pay Raises**: If no employees have a `RateChangeDate` on or after January 1, 2012, the query will return no results.
- **Multiple Pay Changes**: The query will return multiple pay history records if an employee has more than one pay change, sorted by `RateChangeDate`.

### Assumptions

- The `EmployeeDepartmentHistory` table accurately reflects department changes with non-NULL `EndDate` values.
- The `EmployeePayHistory` table includes accurate `RateChangeDate` values for employees who have received pay raises.
- Indexes on `BusinessEntityID` in both tables are assumed for optimal performance.

### Output Example (Columns):

- **BusinessEntityID**: The ID of the employee.
- **RateChangeDate**: The date on which the employee's pay rate changed.
- **Rate**: The new pay rate after the change.
- **PayFrequency**: How often the pay is distributed (e.g., bi-weekly, monthly).

In [31]:
WITH RecentRaises AS
(
    SELECT BusinessEntityID
    FROM HumanResources.EmployeeDepartmentHistory
    WHERE EndDate IS NOT NULL
INTERSECT
    SELECT BusinessEntityID
    FROM HumanResources.EmployeePayHistory
    WHERE RateChangeDate >= '2012-01-01'
)
SELECT EPH.* 
FROM RecentRaises AS RR
    INNER JOIN HumanResources.EmployeePayHistory AS EPH 
        ON RR.BusinessEntityID = EPH.BusinessEntityID
ORDER BY BusinessEntityID, RateChangeDate;

(12 rows affected)

Total execution time: 00:00:00.003

BusinessEntityID,RateChangeDate,Rate,PayFrequency,ModifiedDate
16,2007-12-20 00:00:00.000,24.00,2,2007-12-06 00:00:00.000
16,2009-07-15 00:00:00.000,28.75,2,2009-07-01 00:00:00.000
16,2012-04-30 00:00:00.000,37.50,2,2012-04-16 00:00:00.000
224,2008-01-08 00:00:00.000,8.62,2,2007-12-25 00:00:00.000
224,2011-08-31 00:00:00.000,13.50,2,2011-08-17 00:00:00.000
224,2012-01-01 00:00:00.000,16.00,2,2011-12-18 00:00:00.000
234,2009-01-31 00:00:00.000,39.06,2,2009-01-17 00:00:00.000
234,2011-11-14 00:00:00.000,48.5577,2,2011-10-31 00:00:00.000
234,2012-01-29 00:00:00.000,60.0962,2,2012-01-15 00:00:00.000
250,2011-02-21 00:00:00.000,9.86,2,2011-02-07 00:00:00.000


# Proposition 10: Order of Precedence Exercise

This SQL query retrieves a list of products from the `Production.Product` table by combining results from multiple conditions using `UNION`, `INTERSECT`, and `EXCEPT` clauses. The query returns products that have either a `SellEndDate` or a `ListPrice` of zero, are black in color, and are not classified as 'L' under the `Class` attribute.

### Key Objectives:

1. **Product Selection by SellEndDate and ListPrice**: The query selects products that either have a non-null `SellEndDate` or a `ListPrice` of zero.
2. **Intersection with Black Products**: From this initial set of products, only those that have the color 'Black' are selected.
3. **Exclusion of Class 'L' Products**: Finally, products classified as 'L' in the `Class` attribute are excluded from the final result set.

## Functional Specification

### Inputs

- **Production.Product table**: Contains product information, including:
    - `ProductID`: Unique identifier for the product.
    - `Name`: Name of the product.
    - `SellEndDate`: The date when the product is no longer available for sale.
    - `ListPrice`: The price of the product.
    - `Color`: The color of the product.
    - `Class`: The product's classification (such as 'L', 'M', 'H').

### Outputs

- **ProductID**: Unique identifier for each product.
- **Name**: Name of the product.
- **SellEndDate**: The end date when the product was or will be available for sale.
- **ListPrice**: The list price of the product.
- **Color**: The color of the product.
- **Class**: The classification of the product.

### Query Breakdown

1. **Product Selection by SellEndDate or ListPrice**:
    
    - The first `SELECT` statement retrieves products where `SellEndDate` is not null.
    - The second `SELECT` statement retrieves products where `ListPrice` equals zero.
    - These two result sets are combined using `UNION`, which returns the distinct rows from both queries (no duplicates).
2. **Intersection with Black Products**:
    
    - The `INTERSECT` clause is used to find products from the `UNION` result that are black in color (`Color = 'Black'`). Only products that appear in both the combined `UNION` result and the products that are black are returned.
3. **Exclusion of Class 'L' Products**:
    
    - The `EXCEPT` clause is used to remove any products from the previous result set that are classified as 'L' in the `Class` field.
4. **Final Result**:
    
- The final result set contains products that satisfy all the conditions: either a non-null `SellEndDate` or a `ListPrice` of zero, that are black in color, and are not classified as 'L'.

### Edge Cases

- **Products with Multiple Null Attributes**: Products that meet more than one condition (e.g., having a non-null `SellEndDate` and a `ListPrice` of zero) will only appear once due to the `UNION` operation.
- **No Black Products**: If there are no products that are black in color, the final result set will be empty, regardless of any other conditions.
- **All Products Class 'L'**: If all black products have a `Class` of 'L', the final result set will be empty due to the `EXCEPT` clause.

### Assumptions

- The `Production.Product` table has appropriate indexes on the `SellEndDate`, `ListPrice`, `Color`, and `Class` columns to improve performance, especially for filtering and set operations.
- The business logic requires all products returned to be black in color, and products classified as 'L' should be excluded from the result.

### Considerations

- **Performance**: The use of `UNION`, `INTERSECT`, and `EXCEPT` may involve multiple scans of the `Production.Product` table. For large datasets, optimizing with appropriate indexes on the columns involved (`SellEndDate`, `ListPrice`, `Color`, `Class`) would help improve query performance.

In [32]:

    SELECT ProductID, Name, SellEndDate, ListPrice, Color, Class
    FROM Production.Product
    WHERE SellEndDate IS NOT NULL
UNION
    SELECT ProductID, Name, SellEndDate, ListPrice, Color, Class
    FROM Production.Product
    WHERE ListPrice = 0
INTERSECT
    SELECT ProductID, Name, SellEndDate, ListPrice, Color, Class
    FROM Production.Product
    WHERE Color = N'Black'
EXCEPT
    SELECT ProductID, Name, SellEndDate, ListPrice, Color, Class
    FROM Production.Product
    WHERE Class = N'L'

(77 rows affected)

Total execution time: 00:00:00.006

ProductID,Name,SellEndDate,ListPrice,Color,Class
318,ML Crankarm,NULL,0.00,Black,M
319,HL Crankarm,NULL,0.00,Black,NULL
322,Chainring,NULL,0.00,Black,NULL
709,"Mountain Bike Socks, M",2012-05-29 00:00:00.000,9.50,White,NULL
710,"Mountain Bike Socks, L",2012-05-29 00:00:00.000,9.50,White,NULL
731,"ML Road Frame - Red, 44",2012-05-29 00:00:00.000,594.83,Red,M
732,"ML Road Frame - Red, 48",2012-05-29 00:00:00.000,594.83,Red,M
733,"ML Road Frame - Red, 52",2012-05-29 00:00:00.000,594.83,Red,M
734,"ML Road Frame - Red, 58",2012-05-29 00:00:00.000,594.83,Red,M
735,"ML Road Frame - Red, 60",2012-05-29 00:00:00.000,594.83,Red,M


### Footnote: ChatGPT used to help write only the proposition text.